In [15]:
# %pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="lWv4zzEGXMa2nbwbIa6h")
# project = rf.workspace("dimas-daffa").project("license-vehicle")
# version = project.version(4)
# dataset = version.download("yolov8")
                

In [16]:
import os
from PIL import Image

# Path ke direktori gambar dan anotasi
image_dir = r'D:\license-plate-yolov\license-vehicle-3\train\images'
label_dir = r'D:\license-plate-yolov\license-vehicle-3\train\labels'
output_folder = r'D:\license-plate-yolov\cropped'  # Folder output yang diperbarui

# Pastikan folder output ada
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Fungsi untuk mengubah anotasi YOLO ke koordinat pixel
def yolo_to_bbox(x_center, y_center, width, height, img_width, img_height):
    xmin = int((x_center - width / 2) * img_width)
    ymin = int((y_center - height / 2) * img_height)
    xmax = int((x_center + width / 2) * img_width)
    ymax = int((y_center + height / 2) * img_height)
    return xmin, ymin, xmax, ymax

# Loop melalui setiap file label
for label_filename in os.listdir(label_dir):
    if label_filename.endswith('.txt'):
        # Path file label dan gambar
        label_path = os.path.join(label_dir, label_filename)
        image_path = os.path.join(image_dir, label_filename.replace('.txt', '.jpg'))  # Asumsi gambar berformat .jpg

        # Baca gambar
        image = Image.open(image_path)
        img_width, img_height = image.size

        # Baca anotasi dari file label
        with open(label_path, 'r') as f:
            annotations = f.readlines()

        # Loop melalui setiap anotasi dalam file
        for i, annotation in enumerate(annotations):
            # Ambil data anotasi (class_id, x_center, y_center, width, height)
            parts = annotation.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])

            # Konversi ke koordinat pixel
            xmin, ymin, xmax, ymax = yolo_to_bbox(x_center, y_center, width, height, img_width, img_height)

            # Crop gambar
            cropped_image = image.crop((xmin, ymin, xmax, ymax))

            # Simpan gambar yang sudah dicrop
            output_filename = f"{label_filename.replace('.txt', '')}_cropped_{i}.jpg"
            cropped_image.save(os.path.join(output_folder, output_filename))

print("Proses crop selesai!")


Proses crop selesai!


In [30]:
import os
from PIL import Image

# Path ke direktori gambar dan anotasi
image_dir = r'D:\license-plate-yolov\plat-annot\train\images'
label_dir = r'D:\license-plate-yolov\plat-annot\train\labels'
output_folder = r'D:\license-plate-yolov\plat-cropped'  # Folder output yang diperbarui

# Pastikan folder output ada
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Fungsi untuk mengubah anotasi YOLO ke koordinat pixel
def yolo_to_bbox(x_center, y_center, width, height, img_width, img_height): 
    xmin = int((x_center - width / 2) * img_width)
    ymin = int((y_center - height / 2) * img_height)
    xmax = int((x_center + width / 2) * img_width)
    ymax = int((y_center + height / 2) * img_height)
    return xmin, ymin, xmax, ymax

# Loop melalui setiap file label
for label_filename in os.listdir(label_dir):
    if label_filename.endswith('.txt'):
        # Path file label dan gambar
        label_path = os.path.join(label_dir, label_filename)
        image_path = os.path.join(image_dir, label_filename.replace('.txt', '.jpg'))  # Asumsi gambar berformat .jpg

        # Baca gambar
        image = Image.open(image_path)
        img_width, img_height = image.size

        # Baca anotasi dari file label
        with open(label_path, 'r') as f:
            annotations = f.readlines()

        # Loop melalui setiap anotasi dalam file
        for i, annotation in enumerate(annotations):
            # Ambil data anotasi (class_id, x_center, y_center, width, height)
            parts = annotation.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])

            # Konversi ke koordinat pixel
            xmin, ymin, xmax, ymax = yolo_to_bbox(x_center, y_center, width, height, img_width, img_height)

            # Crop gambar
            cropped_image = image.crop((xmin, ymin, xmax, ymax))

            # Simpan gambar yang sudah dicrop
            output_filename = f"{label_filename.replace('.txt', '')}_cropped_{i}.jpg"
            cropped_image.save(os.path.join(output_folder, output_filename))

print("Proses crop plat selesai!")


Proses crop plat selesai!


In [1]:
# %pip install opencv-contrib-python

import os
from PIL import Image
import cv2 as cv
import numpy as np

# Paths to directories
cropped_input_folder = r'D:\license-plate-yolov\plat-cropped'  # Folder berisi gambar yang sudah dicrop
reso_output_folder = r'D:\license-plate-yolov\reso'            # Folder output untuk gambar super-resolved
model_path = r'D:\license-plate-yolov\EDSR_x4.pb'        # Path ke model super-resolution

# Pastikan folder output ada
if not os.path.exists(reso_output_folder):
    os.makedirs(reso_output_folder)

# Inisialisasi model Super Resolution
sr = cv.dnn_superres.DnnSuperResImpl_create()
sr.readModel(model_path)
sr.setModel('edsr', 4)  # 'edsr' adalah nama model, 4 adalah faktor peningkatan

# Fungsi untuk menerapkan super resolution pada satu gambar
def apply_super_resolution(input_image_path, output_image_path):
    # Buka gambar menggunakan PIL
    image = Image.open(input_image_path)
    
    # Konversi PIL Image ke format OpenCV (BGR)
    img_cv = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)
    
    # Handle gambar grayscale
    if len(img_cv.shape) == 2:
        img_cv = cv.cvtColor(img_cv, cv.COLOR_GRAY2BGR)
    
    # Terapkan Super Resolution
    try:
        sr_img = sr.upsample(img_cv)
    except Exception as e:
        print(f"Super Resolution gagal untuk {input_image_path}: {e}")
        return
    
    # Konversi kembali ke RGB untuk disimpan dengan PIL
    sr_img_rgb = cv.cvtColor(sr_img, cv.COLOR_BGR2RGB)
    sr_pil = Image.fromarray(sr_img_rgb)
    
    # Simpan gambar yang telah ditingkatkan resolusinya
    sr_pil.save(output_image_path)
    print(f"Super-resolved image disimpan di {output_image_path}")

# Loop melalui setiap gambar yang telah dicrop dan terapkan super resolution
for filename in os.listdir(cropped_input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
        input_path = os.path.join(cropped_input_folder, filename)
        output_filename = f"{os.path.splitext(filename)[0]}_reso.jpg"  # Anda dapat mengubah format sesuai kebutuhan
        output_path = os.path.join(reso_output_folder, output_filename)
        
        apply_super_resolution(input_path, output_path)

print("Proses Super Resolution selesai!")


Super-resolved image disimpan di D:\license-plate-yolov\reso\plat-crop-1-_jpg.rf.0c66575900b97c7c2178cc831ce21c10_cropped_0_reso.jpg
Super-resolved image disimpan di D:\license-plate-yolov\reso\plat-crop-1-_jpg.rf.0c66575900b97c7c2178cc831ce21c10_cropped_1_reso.jpg
Super-resolved image disimpan di D:\license-plate-yolov\reso\plat-crop-100-_jpg.rf.ae7b161718034de049fbf2f9ff93048a_cropped_0_reso.jpg
Super-resolved image disimpan di D:\license-plate-yolov\reso\plat-crop-1002-_jpg.rf.a0f5925c4b9d5a950d56aa55949af159_cropped_0_reso.jpg
Super-resolved image disimpan di D:\license-plate-yolov\reso\plat-crop-1003-_jpg.rf.2b3296f4ae467fae0e536e7dd6756d68_cropped_0_reso.jpg
Super-resolved image disimpan di D:\license-plate-yolov\reso\plat-crop-1004-_jpg.rf.bc9865b9495931c02257fc1c5f740a40_cropped_0_reso.jpg
Super-resolved image disimpan di D:\license-plate-yolov\reso\plat-crop-1004-_jpg.rf.bc9865b9495931c02257fc1c5f740a40_cropped_1_reso.jpg
Super-resolved image disimpan di D:\license-plate-yolov

In [ ]:
# %pip install tesseract

     ---------------------------------------- 0.0/45.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/45.6 MB ? eta -:--:--
     ---------------------------------------- 0.1/45.6 MB 2.0 MB/s eta 0:00:24
     ---------------------------------------- 0.4/45.6 MB 4.1 MB/s eta 0:00:11
     ---------------------------------------- 0.4/45.6 MB 4.1 MB/s eta 0:00:11
     ---------------------------------------- 0.4/45.6 MB 2.3 MB/s eta 0:00:20
     ---------------------------------------- 0.4/45.6 MB 2.3 MB/s eta 0:00:20
     ---------------------------------------- 0.4/45.6 MB 1.6 MB/s eta 0:00:29
     ---------------------------------------- 0.5/45.6 MB 1.4 MB/s eta 0:00:32
     ---------------------------------------- 0.5/45.6 MB 1.4 MB/s eta 0:00:33
     ---------------------------------------- 0.5/45.6 MB 1.3 MB/s eta 0:00:36
      --------------------------------------- 0.6/45.6 MB 1.4 MB/s eta 0:00:34
      --------------------------------------- 0.7/45.6 MB 1.5 MB/s


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\Nino\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
%pip install pytesseract

import os  
from PIL import Image  
import pytesseract  

# Path ke folder yang berisi gambar yang sudah dipotong  
cropped_folder = r'D:\license-plate-yolov\reso'  
output_text_file = r'D:\license-plate-yolov\plat-cropped\ocr_results.txt'  

# Pastikan path ke Tesseract sudah sesuai  
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Sesuaikan dengan path instalasi Tesseract Anda  

try:
    # Membuka file output untuk menyimpan hasil OCR  
    with open(output_text_file, 'w') as f:  
        # Loop melalui setiap file gambar di folder cropped  
        for image_filename in os.listdir(cropped_folder):  
            if image_filename.endswith('.jpg'):  
                # Path gambar  
                image_path = os.path.join(cropped_folder, image_filename)  

                # Membaca gambar  
                image = Image.open(image_path)  

                # Melakukan OCR  
                text = pytesseract.image_to_string(image)  

                # Menulis hasil OCR ke file  
                f.write(f"{image_filename}: {text}\n")  

    print("Proses OCR selesai, hasil disimpan di 'ocr_results.txt'.")
except PermissionError as e:
    print(f"PermissionError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Note: you may need to restart the kernel to use updated packages.
Proses OCR selesai, hasil disimpan di 'ocr_results.txt'.


In [3]:
#GAGAL
%pip install imutils
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
import glob

# Path to the folder containing images
image_folder = "D:\license-plate-yolov\\reso\**\*"

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Get a list of all image paths in the folder
image_paths = glob.glob(image_folder, recursive=True)

# List to store detection results
detection_results = []
not_detected_files = []

# Process each image
for image_path in image_paths:
    img = cv2.imread(image_path)
    if img is None:
        print(f"Unable to read image: {image_path}")
        not_detected_files.append(image_path)
        continue

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply filters and edge detection
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
    edged = cv2.Canny(bfilter, 30, 200)  # Edge detection

    # Find contours
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

    # Detect the location of the contour (license plate)
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:  # Looking for a quadrilateral (license plate)
            location = approx
            break

    if location is None:
        not_detected_files.append(image_path)
        continue

    # Create a mask for the detected contour
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0, 255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)

    # Extract the region of interest
    (x, y) = np.where(mask == 255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]

    # Use EasyOCR to read text from the cropped image
    result = reader.readtext(cropped_image)

    if result:
        text = result[0][-2]  # Extract detected text (plate number)
        detection_results.append((image_path, text))
    else:
        not_detected_files.append(image_path)

<>:10: SyntaxWarning: invalid escape sequence '\l'
<>:10: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Nino\AppData\Local\Temp\ipykernel_10204\240202101.py:10: SyntaxWarning: invalid escape sequence '\l'
  image_folder = "D:\license-plate-yolov\\reso\**\*"


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25855 sha256=cd2a692f652808f1405bd767afd0191aab1adb2355cd80bd43a94e3a025c515d
  Stored in directory: c:\users\nino\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [4]:
# GAGAL
# Count detected plates
detected_count = len(detection_results)

# Display results of detected plates
print(f"Total detected plates: {detected_count}")
print("Detected Plates and Their File Paths:")
for file, text in detection_results:
    print(f"File: {file}, Plate: {text}")
    
# Display files with no detections
print("\nFiles with no detections:")
for file in not_detected_files:
    print(f"File: {file}")

Total detected plates: 66
Detected Plates and Their File Paths:
File: D:\license-plate-yolov\reso\plat-crop-118-_jpg.rf.5f5250e3bac61fe35bb8bb85131f80d5_cropped_0_reso.jpg, Plate: 0
File: D:\license-plate-yolov\reso\plat-crop-130-_jpg.rf.464a7154eb333d7b5a6e340f13cd5fac_cropped_0_reso.jpg, Plate: 0
File: D:\license-plate-yolov\reso\plat-crop-144-_jpg.rf.21c22863454fa47537c8c42f7d1213a8_cropped_0_reso.jpg, Plate: 6 ) Ulo BJh
File: D:\license-plate-yolov\reso\plat-crop-191-_jpg.rf.c256f6991c16e626311a23e05072296a_cropped_1_reso.jpg, Plate: P
File: D:\license-plate-yolov\reso\plat-crop-195-_jpg.rf.698d89b8ea03dedbfc4c0d2b2d4d3fe0_cropped_0_reso.jpg, Plate: 6 1570 EvH
File: D:\license-plate-yolov\reso\plat-crop-21-_jpg.rf.8c8af56c7f8e5a16436684d62d4ef0a1_cropped_0_reso.jpg, Plate: B
File: D:\license-plate-yolov\reso\plat-crop-211-_jpg.rf.d754c43904689b2db84074681782fec3_cropped_0_reso.jpg, Plate: E
File: D:\license-plate-yolov\reso\plat-crop-213-_jpg.rf.0e46ceb28a831444427f47a43f1f51d3_cro